step 1: duplication heads, s2 write to itself
step 2: s-inhibition heads, move information from s2 to end
step 3: name mover heads, attend to io from end

In [ ]:
query=padding tokens,
key=non-padding

mask everything else

In [2]:
from transformer_lens import HookedTransformerConfig, HookedTransformer

In [ ]:
config = HookedTransformerConfig(**params)

In [ ]:
HookedTransformer(cfg=config)

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
resid = cache["resid"]
final_resid = resid[:, -1, :]
scaled_final_resid = model.apply_ln_to_stack(final_resid)

In [ ]:
W_U = model.W_U

In [ ]:
logit_diff = scaled_final_resid @ (
    W_U[:, correct_token],
    W_U[:, incorrect_token]
)

In [ ]:
clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

In [ ]:
_, clean_cache = model.run_with_cache(clean_tokens)
_, corrupted_cache = model.run_with_cache(corrupted_tokens)

In [ ]:
correct_token = model.to_single_token(" John")
incorrect_token = model.to_single_token(" Mary")

In [ ]:
seq_len = tokens.shape[-1]

In [6]:
from functools import partial

In [7]:
def patch_act(act, hook, corrupted_activations, position_idx):
    act[:, position_idx, :] = corrupted_activations[hook.name][:, -1, :]
    return act

In [ ]:
def compute_logit_diff(logits, correct_token, incorrect_token):
    final_logits = logits[:, -1, :]
    return final_logits[:, correct_token] - final_logits[:, incorrect_token]

In [8]:
from transformer_lens.utils import get_act_name

In [ ]:
results = torch.zeros(n_layers, seq_len)

for layer_idx in range(n_layers):
    hook_name = get_act_name("resid_pre", layer_idx)
    
    for position_idx in range(seq_len):
        hook_func = partial(
            patch_act,
            corrupted_activations=corrupted_activations,
            position_idx=position_idx
        )
        
        patched_logits, _ = model.run_with_cache(
            clean_tokens
        )
        
        result[layer_idx, position_idx] = compute_logit_diff(
            patched_logits,
            correct_token,
            incorrect_token
        )

In [ ]:
step 1: project to fourier basis
step 2: extract frequenciies

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
W_in_acts = cache["post", 2]
W_out = model.w_out[2]

output = W_in_acts @ W_out

In [ ]:
A @ x @ W_OV @ W_OV

In [ ]:
_, clean_cache = model.run_with_cache(clean_tokens)
_, corrupted_cache = model.run_with_cache(corrupted_tokens)

In [9]:
def patch_head(act, hook, clean_cache, corrupted_cache, target_head):
    target_layer_idx, target_head_idx = target_head
    if target_layer_idx == hook.layer():
        act[:, :, head_idx] = corrupted_cache[hook.name][:, :, head_idx]
    else:
        act = clean_cache
    
    return act

In [10]:
from itertools import product

In [ ]:
combinations = product(range(n_layers), range(n_heads))

In [ ]:
results = torch.zeros(n_layers, n_heads)

for layer_idx, head_idx in combinations:
    model.reset_hooks()
    hook_name = get_act_name("z", layer_idx)
    hook_func = partial(
        patch_head,
        clean_cache=clean_cache,
        corrupted_cache,
        target_head=(layer_idx, head_idx)
    )
    
    model.add_hook(hook_name, hook_func)
    patched_logits, _ = model.run_with_cache(
        clean_tokens
    )
    
    results[layer_idx, head_idx] = compute_ioi_metric(patched_logits)

In [ ]:
empty = linear_probe[..., 0]
mine = linear_probe[..., 1]
theirs = linear_probe[..., 2]

not_empty = (mine - theirs) / 2

In [ ]:
empty_vs_not_empty = empty - not_empty

In [ ]:
step 1: logit lens across residual stream
step 2: logit lens across decomposed residual stream
step 3: logit lens across decomposed attn layer

In [11]:
from transformer_lens import HookedTransformerConfig

In [ ]:
cfg = HookedTransformerConfig(**params)
model = HookedTransformer(cfg=cfg)

In [ ]:
[v0, v1] @ W_E @ W_K @ W_Q @ W_E @ [v_2]

In [12]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class ParallelEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, parallel_context):
        super().__init__()
        world_size = parallel_context.get_world_size(ParallelMode.TENSOR)
        per_partition = num_embeddings // world_size
        
        self.weight = nn.Parameter(torch.randn(
            per_partition,
            embedding_dim
        ))
        self.vocab_start_idx, self.vocab_end_idx = self._get_vocab_range(
            per_partition, parallel_context
        )
    
    def _get_vocab_range(self, per_partition, parallel_context):
        rank = parallel_context.get_local_rank(ParallelMode.TENSOR)
        start_idx = rank*per_partition
        end_idx = start_idx + per_partition
        return start_idx, end_idx

    def forward(self, input):
        input_mask = (input < self.vocab_start_idx) | (input >= self.vocab_end_idx)
        masked_input = input[input_mask]

In [ ]:
broadcast, gather

In [ ]:
broadcast > gather > scatter > all-reduce

In [13]:
import torch.distributed as dist

In [17]:
class Scatter(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = dist.get_world_size()
        rank = dist.get_rank()
        inputs = torch.split(
            input,
            split_size_or_sections=input.shape[-1]//world_size
        )
        return inputs[rank]

    @staticmethod
    def backward(ctx, grad_input):
        world_size = dist.get_world_size()
        grads = [torch.zeros_like(grad_input) for _ in range(world_size)]
        dist.all_gather(grads, grad_input)
        grads = torch.cat(grads)
        return grads

In [16]:
class Reduce(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        dist.all_reduce(input)
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        return (grad_input, None)

In [14]:
class RowParallelLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        world_size = dist.get_world_size()
        input_per_partition = output_size // world_size
        
        self.weight = nn.Parameter(torch.randn(
            output_size,
            input_per_partition
        ))
        self.bias = nn.Parameter(torch.randn(
            output_size
        ))
    
    def forward(self, input):
        input_parallel = Scatter.apply(input)
        output_parallel = F.linear(input_parallel, self.weight)
        outputs = Reduce.apply(input_parlalel)
        return outputs + self.bias

In [19]:
class Reduce(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, parallel_context):
        group = parallel_context.get_group(ParallelMode.TENSOR)
        dist.all_reduce(input, group=group)
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        return (grad_input, None)

In [18]:
class ParallelEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, parallel_context):
        super().__init__()
        world_size = dist.get_world_size()
        per_partition = num_embeddings // world_size
        self.weight = nn.Parameter(torch.randn(
            per_partition,
            embedding_dim
        ))
        self.vocab_start_idx, self.vocab_end_idx = self._get_vocab_range(
            per_partition,
            parallel_context
        )
    
    def _get_vocab_range(self, per_partition, parallel_context):
        rank = parallel_context.get_local_rank(ParallelMode.TENSOR)
        start_idx = rank*per_partition
        end_idx = start_idx+per_partition
        return start_idx, end_idx

    def forward(self, input):
        input_mask = (input < self.vocab_start_idx) | (input >= self.vocab_end_idx)
        masked_input = input.clone() - self.vocab_start_idx
        masked_input[input_mask] = 0
        
        parallel_embedding = F.embedding(masked_input, self.weight)
        parallel_embedding[input_mask, :] = 0.
        
        embeddings = Reduce.apply(parallel_embedding, parallel_context)
        return embeddings

In [ ]:
Scatter > All-reduce > Identity > Gather

In [ ]:
hostdiscovery, elasticdriver, 3 notifications, torchstate, 

In [ ]:
step 1: num_processed=0
step 2: unprocess
step 3: work nodes
step 4: get unprocessed samples

In [ ]:
class Recompute(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.function = function
        ctx.input = input
        ctx.recomputed = recomputed
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        input_leaf = self.input.detach().requires_grad_(
            input.requires_grad
        )
        
        with torch.enable_grad():
            outputs = self.function(input_leaf)
        
        ctx.recomputed.append((outputs, input_leaf))
        

tensor_parallel_size, pipeline_parallel_size, data_parallel_size, world_size, rank

In [ ]:
(local_rank+1) % world_size

In [ ]:
step 1: send metadata
step 2: send data
step 3: receive the metadata, reconstruct 
step 4: fill

In [20]:
class Checkpoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.recomputed = recomputed
        ctx.function = function
        
        with torch.no_grad():
            output = function(input)
        
        return output
    
    @staticmethod
    def backward(ctx, grad_input):
        output, input_leaf = ctx.recompute.pop()
        
        with torch.enable_grad():
            torch.autograd.backward(output, grad_input)
        
        return tuple([None, None, None, input_leaf.grad])

In [ ]:
shape, dtype, requires_grad

In [ ]:
rank = dist.get_rank()

In [ ]:
if rank == 69:
    dist.isend(x, dst=42)
elif rank == 42:
    dist.irecv(tensor_will_be_received_data, x, src=69)

In [ ]:
class Recompute(torch.autograd.Function):
    @staticmethod
    def forward(ctx, )

In [ ]:
class Checkpoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.phony = phony
        ctx.recomputed = recomputed
        
        with torch.no_grad():
            output = function(input)
        
        return output
    
    @staticmethod
    def backward(ctx, grad_input):
        output, input_leaf = ctx.recomputed.pop()
        
        with torch.enable_grad():
            torch.autograd.backward(output, grad_input)
        
        return tuple([None, None, None, input_leaf.grad])

In [ ]:
step 1: create a mapping between datatype and integer
step 2: convert
step 3: send
step $

In [ ]:
register > cache > sram > disk

In [ ]:
class Recompute(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.recomputed = recomputed
        ctx.function = function
        ctx.input = input
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        function = ctx.function
        input_leaf = ctx.input.detach().requires_grad(
            ctx.input.requires_grad
        )
        
        with torch.no_grad():
            output = function(input_leaf)
            
        ctx.recomputed.append((output, input_leaf ))
        
        return tuple([None, None, None, grad_input])

In [ ]:
A @ x @ W_OV^{1} @ W_OV^{2}

step 1: logits = resid @ unembed
step 2: resid = embed + pos_embed + sum(12 heads)
step 3: logits = [embed + pos_embed + sum(12 heads)] @ unembed
step 4: logits = embed @ unembed + pos_embed @ unembed + sum(12 heads) @ unembed

In [ ]:
relu
correlated vs anti-correlated
feature importance
feature sparity

step 1: a clean prompt, a corrupted prompt
step 2: record the interdimate actiavtions of the clean prompt and corrupted prompt
step 3: run the clean prompt again, if a component is the target source head, or an mlp in between, then patch the activations from the corrupted prompt, otherwise, patch from the clean prompt
step 4: continue

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
input_components = torch.cat([
    cache["embed"],
    cache["pos_embed"],
    cache["result", layer_idx-1]
])

In [21]:
from einops import einsum

In [ ]:
W_Q = model.W_Q[layer_idx, head_idx]
decomposed_query = einsum(
    input_components,
    W_Q
)

In [ ]:
W_K = model.W_K[layer_idx]
decomposed_key = einsum(
    input_components,
    W_K
)

In [ ]:
decomposed_scores = einsum(
    decomposed_query,
    decomposed_key
)

In [ ]:
step 1: x @ W_Q
step 2: x = embed + pos_embed + sum(12 heads)
step 3: 

In [ ]:
conditioned response, stimulus-evoked, motor imagery, populaition

In [ ]:
activation ranges: 0.42 - 0.69
percentage 2%
met hypothesis

In [ ]:
duplication head, name mover heads, s-inhibition heads

In [ ]:
neuron fire, but not the target pattern

In [ ]:
local_rank, process_group, ranks_in_group, local_world_size

In [24]:
import torch.distributed.rpc as rpc
from torch.distributed.rpc import RRef

In [25]:
class Agent:
    def __init__(self):
        self.id = RRef(self)

In [ ]:
encodings.masked_fill(mask == False, -1)

In [26]:
from torch.distributed.rpc import RRef

In [27]:
class Agent:
    def __init__(self):
        self.id = RRef(self)

In [ ]:
biocompabile,
reliable recording
neural plasiticity

In [ ]:
def discount_reward(rewards, discount_factor):
    x = torch.pow(discount_factor, len(rewards))
    return rewards*x 

In [28]:
from einops import rearrange

In [ ]:
output = rearrange(
    images,
    "bs c (p_height n_height) (p_width n_width) -> \
    bs (n_height n_width) (p_height p_width c)
    "
)

In [ ]:
training task: training distribution, objective
base optimizer: sgd, architecture

In [31]:
rank = 2

In [29]:
ranks = [1, 2, 3, 4, 5]

In [ ]:
for other in ranks:
    if other == rank:
        continue

In [34]:
name = f"worker_"